## 3. Word Similarity Task

In [1]:
from scipy.spatial.distance import cosine
import pandas as pd
import csv
import numpy as np

#f = np.genfromtxt('deps.words')
def readembeddings(fn):
    e=[]
    l=[]
    d=dict()
    df = pd.DataFrame()
    with open(fn, 'rb') as f:
        data = f.readlines()
        i=0
        for row in data:
            l=[item.decode("utf-8")  for item in row.split()]
            t=(l[0],l[1:])
            e.append(t)
            i+=1
        df = pd.DataFrame(e)
        df.columns=['word','embeddings']
        return df
        

deps = readembeddings('deps.words') 
bow2 = readembeddings('bow2.words')
bow5 = readembeddings('bow5.words')




In [4]:
def cosine_similarity(v1, v2):
    #print(v1[0])
    #print(v2[0])
    v1=np.array(v1[0], dtype=float)
    v2=np.array(v2[0], dtype=float)

    cos = np.dot(v1, v2)/(np.linalg.norm(v1)* np.linalg.norm(v2))
    return cos

def filterwords(e, simlex, model):
    cos = []
    #df=pd.DataFrame(columns=['w1', 'w2', 'simlex999','cosine'])
    for w1, w2 in zip(simlex['word1'].values, simlex['word2'].values):
    #print('type word', type(words))
        #print(list(words))
        if w1 in e['word'].values and w2 in e['word'].values:
            #print('found')
            v1=e[e['word']==w1]
            v2=e[e['word']==w2]
            cos.append(cosine_similarity(v1['embeddings'].values, v2['embeddings'].values))
        else:
            print('word 1=',w1, 'word 2=', w2)
            cos.append(-100)
    
    simlex[model] = cos
    return simlex

        
#cos_b2=deps[deps['word']=='the']
#x=[float(n) for n in a['embeddings'].values[0]]
#print(x)

df_simlex = pd.read_csv('SimLex-999.txt', delimiter='\t')
df_simlex.head()
#l1 = [[x,y] for x, y in zip(df_simlex['word1'],df_simlex['word2'])]
#print(l1)



i=0
prod=[]
model='deps'
print('model=',model)
df_simlex[model] = 0
df_cosine= filterwords(deps, df_simlex, 'deps')
model='bow2'
print('model=',model)
df_simlex[model] = 0
df_cosine= filterwords(bow2, df_simlex, 'bow2')
model='bow5'
print('model=',model)
df_simlex[model] = 0
df_cosine= filterwords(bow5, df_simlex, 'bow5')
#print(len(f_deps))


model= deps
word 1= disorganize word 2= organize
model= bow2
word 1= disorganize word 2= organize
model= bow5
word 1= disorganize word 2= organize


In [44]:
import math

def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    return diffprod / math.sqrt(xdiff2 * ydiff2)
df_cosine.head()
df_cosine=df_cosine[df_cosine['deps'] !=-100]
print(pearson_def(df_cosine['deps'].values, df_cosine['SimLex999'].values))
print(pearson_def(df_cosine['bow2'].values, df_cosine['SimLex999'].values))
print(pearson_def(df_cosine['bow5'].values, df_cosine['SimLex999'].values))




0.461901344297
0.428458661816
0.375600597067


## 4. Word Analogy Task

In [19]:
import numpy as np
from sklearn import preprocessing

data = np.array([[1.0, 1.0, 0.0],
                 [1.0, 1.0, 1.0],
                 [1.0, 2.0, 3.0]],dtype=float)

#n_normalize(data, norm='l2')

def load_questions(fn='questions-words_mini_1.txt'):
    with open(fn, 'rb') as f:
        data = f.readlines()
        i=0
        category=''
        e=[]
        for row in data:
#            print('i=',i, 'row:', row, 'zero:', row[0].decode("utf-8"))
#            if row[0]==':':
#                category=row[2:]
#            else:
            l=[item.decode("utf-8")  for item in row.split()]
            if l[0] == ':':
                category=l[1:]             
                #print('category=',l[1:])
            else:
                t=(category[0], l[0],l[1],l[2], l[3])
                e.append(t)
            i+=1
        df = pd.DataFrame(e)
        df.columns=['category', 'w1','w2','w3', 'w4']
        return df



In [22]:
from sklearn import preprocessing
import pandas as pd
import pickle



#data = np.array([[1.0, 1.0, 0.0],
#                 [1.0, 1.0, 1.0],
#                 [1.0, 2.0, 3.0]],dtype=float)

#n_normalize(data, norm='l2')

def vector_offset(e, df, model):
    def getv(w):
            v =e[e['word']==w.lower()]['embeddings']
#            print('type(v)=', type(v))
            #v=np.array(v.values[0], dtype=float)
            #print(v)
            return v # np.reshape(v, (1,300))
    offset = []
    for w1, w2, w3 in zip(df['w1'].values, df['w2'].values, df['w3'].values):
        #print('w1=',w1)
        #print('w2=',w2)
        #print('w3=',w3)
        w1 = getv(w1)
        w2 = getv(w2)
        w3 = getv(w3)
        #print('w1=',w1)
        #print('w2=',w2)
        #print('w3=',w3)
        #ry:
        w1 = preprocessing.normalize(np.reshape(np.array(w1.values[0], dtype=float), (1,300)), norm='l2')
        w2 = preprocessing.normalize(np.reshape(np.array(w2.values[0], dtype=float), (1,300)), norm='l2')
        w3 = preprocessing.normalize(np.reshape(np.array(w3.values[0], dtype=float), (1,300)), norm='l2')
            #print('w3=', w3, 'w2=', w2, 'w1=',w1)
        offset.append(w3+ w2-w1)
        #except:
        #    offset.append(np.zeros((1,300)))
        #    print('error')
    return offset

print('loading questions')
df = load_questions()
print('compute vector offset')
offset = vector_offset(deps, df, 'deps')
#print('offset=', offset)


loading questions
compute vector offset


In [25]:
def cosine_similarity(v1, v2):
    #print(v1[0])
    #print(v2[0])
    #v1=np.array(v1[0], dtype=float)
    #v2=np.array(v2[0], dtype=float)
    cos = np.dot(v1, v2.T)/(np.linalg.norm(v1)* np.linalg.norm(v2))
    return cos

def cosine_distance(offset, model):
    
    output = []
    i=0
    #m=np.array([np.array(x, dtype=float) for x in model['embeddings'].values])  
    for o in offset:
        #print('shape o=', o.shape)
        distance = []
        #print(len(model['embeddings'].values))
        #print('shape o=',o.shape)
        #o=np.repeat(np.reshape(o, (1,300)), len(model['embeddings'].values), axis=0)
        #print('o=',o)
        #print(model['embeddings'].values)
        print('i=', i)
        i+=1
        #print('shape model=', m.shape)
        for e in model['embeddings'].values:
            #print('e=', e)
            c=cosine_similarity(np.reshape(o, (1,300)),\
                                                       np.reshape(np.array(e, dtype=float), (1,300)))
            d=1-c
            #print('d=', d[0]) 
#            print('distance=', d.shape)
            distance.append(d)
            
        #print('distance=', distance)
        output.append(np.argmin(np.array(distance)))
    
    
    return output

print('cosine distance')
#print('offset=', offset)
distance =cosine_distance(offset, deps)
#print(distance)

with open('outfile', 'wb') as fp:
    pickle.dump(distance, fp)

cosine distance
i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 20
i= 21
i= 22
i= 23
i= 24
i= 25
i= 26
i= 27
i= 28
i= 29
i= 30
i= 31
i= 32
i= 33
i= 34
i= 35
i= 36
i= 37
i= 38
i= 39
i= 40
i= 41
i= 42
i= 43
i= 44
i= 45
i= 46
i= 47
i= 48
i= 49
i= 50
i= 51
i= 52
i= 53
i= 54
i= 55
i= 56
i= 57
i= 58
i= 59
i= 60
i= 61
i= 62
i= 63
i= 64
i= 65
i= 66
i= 67
i= 68
i= 69
i= 70
i= 71
i= 72
i= 73
i= 74
i= 75
i= 76
i= 77
i= 78
i= 79
i= 80
i= 81
i= 82
i= 83
i= 84
i= 85
i= 86
i= 87
i= 88
i= 89
i= 90
i= 91
i= 92
i= 93
i= 94
i= 95
i= 96
i= 97
i= 98
i= 99
i= 100
i= 101
i= 102
i= 103
i= 104
i= 105
i= 106
i= 107
i= 108
i= 109
i= 110
i= 111
i= 112
i= 113
i= 114
i= 115
i= 116
i= 117
i= 118
i= 119
i= 120
i= 121
i= 122
i= 123
i= 124
i= 125
i= 126
i= 127
i= 128
i= 129
i= 130
i= 131
i= 132
i= 133
i= 134
i= 135
i= 136
i= 137
i= 138
i= 139
i= 140
i= 141
i= 142
i= 143
i= 144
i= 145
i= 146
i= 147
i= 148
i= 149
[10851, 11299, 3227, 2663, 13420, 3227, 1120

In [40]:
def get_mrr(distance, df, model):
    def get_idx(word, model):
        
        m = model[model['word']==word.lower()]
        idx = m.index.values[0]
        return idx

    l_idx = []
    l_cat = []
    df_out = pd.DataFrame()
    for w, c in zip(df['w4'].values, df['category']):
        l_idx.append(get_idx(w, model))
        l_cat.append(c)
    i = 0
    ranks = []
    equal = []
    for d, i in zip(distance, l_idx):
        if d == i:
            r = 1/(d+1)            
            equal.append(True)    
        else: 
            r=0
            equal.append(False)    
        ranks.append(r) 
    mrr = np.mean(np.array(ranks, dtype=float))
    df_out['Guess'] = distance
    df_out['Real'] = l_idx
    df_out['Equal'] = equal
    df_out['Category'] = l_cat
    print(df_out[df_out['Equal']==True])
    #print('distance=', distance, 'indexes=', l_idx, 'MRR=', mrr)    
    return mrr

mrr = get_mrr(distance, df, deps)
print('MRR=',mrr)

     Guess   Real  Equal            Category
76   11912  11912   True   gram3-comparative
79    9783   9783   True   gram3-comparative
80    4451   4451   True   gram3-comparative
91    4018   4018   True   gram4-superlative
141  14612  14612   True  gram9-plural-verbs
144   7620   7620   True  gram9-plural-verbs
MRR= 5.72822970008e-06


In [39]:
offset_bow2 = vector_offset(bow2, df, 'bow2')
distance_bow2 =cosine_distance(offset_bow2, bow2)



i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 20
i= 21
i= 22
i= 23
i= 24
i= 25
i= 26
i= 27
i= 28
i= 29
i= 30
i= 31
i= 32
i= 33
i= 34
i= 35
i= 36
i= 37
i= 38
i= 39
i= 40
i= 41
i= 42
i= 43
i= 44
i= 45
i= 46
i= 47
i= 48
i= 49
i= 50
i= 51
i= 52
i= 53
i= 54
i= 55
i= 56
i= 57
i= 58
i= 59
i= 60
i= 61
i= 62
i= 63
i= 64
i= 65
i= 66
i= 67
i= 68
i= 69
i= 70
i= 71
i= 72
i= 73
i= 74
i= 75
i= 76
i= 77
i= 78
i= 79
i= 80
i= 81
i= 82
i= 83
i= 84
i= 85
i= 86
i= 87
i= 88
i= 89
i= 90
i= 91
i= 92
i= 93
i= 94
i= 95
i= 96
i= 97
i= 98
i= 99
i= 100
i= 101
i= 102
i= 103
i= 104
i= 105
i= 106
i= 107
i= 108
i= 109
i= 110
i= 111
i= 112
i= 113
i= 114
i= 115
i= 116
i= 117
i= 118
i= 119
i= 120
i= 121
i= 122
i= 123
i= 124
i= 125
i= 126
i= 127
i= 128
i= 129
i= 130
i= 131
i= 132
i= 133
i= 134
i= 135
i= 136
i= 137
i= 138
i= 139
i= 140
i= 141
i= 142
i= 143
i= 144
i= 145
i= 146
i= 147
i= 148
i= 149


NameError: name 'get_mrr' is not defined

In [42]:
mrr_bow2 = get_mrr(distance_bow2, df, bow2)
print('MRR=',mrr_bow2)

     Guess   Real  Equal                     Category
1     3860   3860   True     capital-common-countries
2      688    688   True     capital-common-countries
5     2483   2483   True     capital-common-countries
80    3383   3383   True            gram3-comparative
106   3302   3302   True  gram6-nationality-adjective
108   3104   3104   True  gram6-nationality-adjective
110  16700  16700   True  gram6-nationality-adjective
112    808    808   True  gram6-nationality-adjective
113   8635   8635   True  gram6-nationality-adjective
MRR= 2.96336336125e-05


In [43]:
offset_bow5 = vector_offset(bow5, df, 'bow5')
distance_bow5 =cosine_distance(offset_bow5, bow5)
mrr_bow5 = get_mrr(distance_bow5, df, bow5)
print('MRR=',mrr_bow5)

i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 20
i= 21
i= 22
i= 23
i= 24
i= 25
i= 26
i= 27
i= 28
i= 29
i= 30
i= 31
i= 32
i= 33
i= 34
i= 35
i= 36
i= 37
i= 38
i= 39
i= 40
i= 41
i= 42
i= 43
i= 44
i= 45
i= 46
i= 47
i= 48
i= 49
i= 50
i= 51
i= 52
i= 53
i= 54
i= 55
i= 56
i= 57
i= 58
i= 59
i= 60
i= 61
i= 62
i= 63
i= 64
i= 65
i= 66
i= 67
i= 68
i= 69
i= 70
i= 71
i= 72
i= 73
i= 74
i= 75
i= 76
i= 77
i= 78
i= 79
i= 80
i= 81
i= 82
i= 83
i= 84
i= 85
i= 86
i= 87
i= 88
i= 89
i= 90
i= 91
i= 92
i= 93
i= 94
i= 95
i= 96
i= 97
i= 98
i= 99
i= 100
i= 101
i= 102
i= 103
i= 104
i= 105
i= 106
i= 107
i= 108
i= 109
i= 110
i= 111
i= 112
i= 113
i= 114
i= 115
i= 116
i= 117
i= 118
i= 119
i= 120
i= 121
i= 122
i= 123
i= 124
i= 125
i= 126
i= 127
i= 128
i= 129
i= 130
i= 131
i= 132
i= 133
i= 134
i= 135
i= 136
i= 137
i= 138
i= 139
i= 140
i= 141
i= 142
i= 143
i= 144
i= 145
i= 146
i= 147
i= 148
i= 149
     Guess   Real  Equal                     Category
1     